In [1]:
import geopandas as gpd
import numpy as np

**CONTENIDO**

- Cargar data CS
- Resultados Complejos observables en CS
- Procesamiento para datasets geograficos
    - Mapas Ejemplo (Base Personas)
    - Comandos Mapbox CLI

# Cargar data CS

### Precios Corrientes

In [2]:
from datetime import datetime
import pandas as pd

# Cargar el índice de precios al consumidor (CPI) desde la fuente
cpi = pd.read_csv('https://raw.githubusercontent.com/matuteiglesias/IPC-Argentina/main/data/info/indice_precios_M.csv', index_col=0)
cpi.index = pd.to_datetime(cpi.index)

# Obtener la fecha de hoy en formato año-mes
hoy = datetime.today().strftime('%Y-%m')

# Calcular el ratio de precios de hoy con respecto a los precios con índice en base al modelo
ix = cpi.loc[hoy, 'index'].values[0] / cpi.loc['2016-01', 'index'].values[0]

# Lista de columnas relacionadas con montos en pesos
columnas_pesos = ['P47T_persona', 'P47T_hogar', 'CBA', 'gap_indigencia', 'CBT', 'gap_pobreza']


## Generar data Cross Section

In [3]:
experiment_tag = 'ARG'
frac = 0.05
path = './../data/Pobreza/'

In [4]:
# ./../data/Pobreza/personas_ingresos_f0.05_2022-05-15_ARG.csv 

## Funcion Sintetizar

In [5]:
import datetime as dt

def sintetizar(data, grouper, base='Personas', frac=0.05):
    df = data.copy()
    df['Total'] = True
    df['timestamp'] = dt.datetime.today()
    df['AGLO_si'] = df.AGLOMERADO != 0
    df['Total_pais'] = True

    # Columnas comunes
    columns_to_groupby = ['Total', 'Pobreza', 'Indigencia']
    
    # Columnas específicas según base
    if 'P47T_persona' in df.columns:
        columns_to_groupby.append('P47T_persona')
    if 'P47T_hogar' in df.columns:
        columns_to_groupby.append('P47T_hogar')
    if 'CB_EQUIV' in df.columns:
        columns_to_groupby.extend(['CB_EQUIV', 'CBA', 'gap_indigencia', 'CBT', 'gap_pobreza'])

    agg_dict = {
        'Total': ['mean', 'sum'],
        'Pobreza': ['mean', 'sum'],
        'Indigencia': ['mean', 'sum']
    }
    
    if 'P47T_persona' in df.columns:
        agg_dict['P47T_persona'] = ['mean', q10, q25, 'median', q75, q90]
    if 'P47T_hogar' in df.columns:
        agg_dict['P47T_hogar'] = ['mean', q10, q25, 'median', q75, q90]
    if 'CB_EQUIV' in df.columns:
        agg_dict['CB_EQUIV'] = ['mean', 'median']
        agg_dict['CBA'] = ['mean', 'median']
        agg_dict['gap_indigencia'] = ['mean', 'median']
        agg_dict['CBT'] = ['mean', 'median']
        agg_dict['gap_pobreza'] = ['mean', 'median']

    df = df.groupby(grouper + ['timestamp'])[columns_to_groupby].agg(agg_dict)
    
    for col in ['Total', 'Pobreza', 'Indigencia']:
        df[(col, 'sum')] = (df[(col, 'sum')]/frac).round(1)
        df[(col, 'mean')] = df[(col, 'mean')].round(4)

    if 'P47T_persona' in df.columns:
        # display(df['P47T_persona'].count())
        df['P47T_persona'] = df['P47T_persona'].round(-1).astype(int)
    if 'P47T_hogar' in df.columns:
        df['P47T_hogar'] = df['P47T_hogar'].round(-1).astype(int)

    agg_result = df.T.set_index(np.repeat(base, df.shape[1]), append=True)
    stacker_ix = [-i for i in range(len(grouper) + 1)]
    agg_result = agg_result.stack(level=stacker_ix).reset_index()
    
    agg_result = agg_result.rename(columns = {'level_0': 'observable', 'level_1': 'sintetico', 'level_2': 'base', 0: 'valor'})
    agg_result['frac'] = frac
    return agg_result

# Funciones de percentil
def q10(x):
    return x.quantile(0.1)

def q25(x):
    return x.quantile(0.25)

def q75(x):
    return x.quantile(0.75)

def q90(x):
    return x.quantile(0.9)


### Unir NOMDPTO y Region

In [6]:

# data = pd.read_csv('./../data/Pobreza/pobreza_'+str(frac)+'_2020.csv')
# /media/matias/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_0.01_2021-2022.csv saved

radio_ref = pd.read_csv('./../data/info/radio_ref.csv')
radio_ref['COD_2010'] = radio_ref['radio'].astype(str).str.zfill(9)
dpto_region = pd.read_csv('./../data/info/DPTO_PROV_Region.csv')
radio_ref = radio_ref.merge(dpto_region)
aglo_labels = pd.read_csv('./../data/info/aglo_labels.csv')[['AGLOMERADO', 'NOMAGLO']]
radio_ref = radio_ref[['RADIO_REF_ID', 'COD_2010', 'IDFRAC', 'NOMDPTO', 'AGLOMERADO', 'Region']].drop_duplicates().merge(aglo_labels)


In [7]:
# data = pd.read_csv('/media/matias/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_0.02_ARGCSactual.csv', encoding_errors='ignore')
#                     # /media/matias/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_0.02_ARGCSactual.csv
# ## Deflacta a precios actuales

# Lista de fechas Q que se van a procesar
Qs = ['2022-05-15', '2022-08-15', '2022-11-15', '2023-02-15']

# Listas para almacenar los datos consolidados de cada Q
all_info_personas = []
all_info_hogares = []

# Bucle para procesar cada valor de Q
for Q in Qs:
    print(f"Procesando fecha Q: {Q}")
    
    # Nombres de los archivos dependientes de Q
    personas_ingresos_Q_file = f'{path}personas_ingresos_f{frac}_{Q}_{experiment_tag}.csv'
    pobreza_hogares_file = f'{path}pobreza_hogares_f{frac}_q{Q}.csv'
    hogares_geo_file = f'{path}hogares_geo_f{frac}_{Q.split("-")[0]}_{experiment_tag}.csv'

    # Cargar los archivos
    personas_ingresos_Q = pd.read_csv(personas_ingresos_Q_file)
    pobreza_hogares = pd.read_csv(pobreza_hogares_file)
    hogares_geo = pd.read_csv(hogares_geo_file)

    # Fusiones para obtener los datasets consolidados
    info_personas = personas_ingresos_Q.merge(pobreza_hogares, on=['HOGAR_REF_ID', 'Q'], how='left').merge(hogares_geo, on='HOGAR_REF_ID', how='left')
    info_hogares = pobreza_hogares.merge(hogares_geo, on='HOGAR_REF_ID', how='left')
    
    # Añadir los conjuntos de datos consolidados a las listas
    all_info_personas.append(info_personas)
    all_info_hogares.append(info_hogares)


Procesando fecha Q: 2022-05-15


/tmp/ipykernel_378716/2008130523.py:24: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  hogares_geo = pd.read_csv(hogares_geo_file)


Procesando fecha Q: 2022-08-15


/tmp/ipykernel_378716/2008130523.py:24: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  hogares_geo = pd.read_csv(hogares_geo_file)


Procesando fecha Q: 2022-11-15


/tmp/ipykernel_378716/2008130523.py:24: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  hogares_geo = pd.read_csv(hogares_geo_file)


Procesando fecha Q: 2023-02-15


/tmp/ipykernel_378716/2008130523.py:24: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  hogares_geo = pd.read_csv(hogares_geo_file)


In [8]:
 
# Concatenar todos los conjuntos de datos de las diferentes fechas
cross_section_personas = pd.concat(all_info_personas, ignore_index=True)
cross_section_hogares = pd.concat(all_info_hogares, ignore_index=True)


## Adaptar datos (Pesos actuales, AGLOS si, IDFRAC)
for df in [cross_section_personas, cross_section_hogares]:
    for col in columnas_pesos:
        if col in df.columns: 
            if col == 'P47T_persona': df[col] = np.power(10, df[col]) - 1
            df[col] = (ix*df[col]).round(-1).astype(int)
            
    # df = df.merge(radio_ref, axis = 1), on = ['RADIO_REF_ID', 'AGLOMERADO'], how = 'left')
    df['AGLO_si'] = df.AGLOMERADO != 0

    df['IDFRAC'] = df['COD_2010'].astype(str).str.zfill(9).str[:-2] + '00'



In [9]:
cross_section_personas[columnas_pesos].describe()

,P47T_persona,P47T_hogar,CBA,gap_indigencia,CBT,gap_pobreza
count,9.151835e+06,9.151835e+06,9.151835e+06,9.151835e+06,9.151835e+06,9.151835e+06
mean,1.165593e+05,6.561225e+05,1.669731e+05,4.891493e+05,3.845804e+05,2.715421e+05
std,1.674738e+05,6.370943e+06,7.694635e+05,5.657031e+06,1.846687e+06,4.681049e+06
min,0.000000e+00,0.000000e+00,1.739000e+04,-2.824670e+06,3.887000e+04,-7.445870e+06
25%,0.000000e+00,2.083100e+05,9.104000e+04,8.436000e+04,2.075600e+05,-8.365000e+04
50%,7.477000e+04,3.498700e+05,1.287200e+05,2.172500e+05,2.949500e+05,6.412000e+04
75%,1.793900e+05,5.507500e+05,1.730700e+05,4.096700e+05,3.959900e+05,2.446800e+05
max,9.567140e+06,4.056016e+08,4.417477e+07,3.766584e+08,1.063836e+08,3.358785e+08


In [10]:
cross_section_hogares[columnas_pesos[1:]].describe()

,P47T_hogar,CBA,gap_indigencia,CBT,gap_pobreza
count,2.765253e+06,2.765253e+06,2.765253e+06,2.765253e+06,2.765253e+06
mean,3.857625e+05,1.045146e+05,2.812477e+05,2.395852e+05,1.461773e+05
std,5.879316e+05,8.355207e+04,5.368206e+05,1.956441e+05,4.792502e+05
min,0.000000e+00,1.739000e+04,-2.824670e+06,3.887000e+04,-7.445870e+06
25%,1.798700e+05,6.123000e+04,9.127000e+04,1.389900e+05,-2.629000e+04
50%,3.049800e+05,9.583000e+04,2.060700e+05,2.197200e+05,9.357000e+04
75%,4.959300e+05,1.376900e+05,3.828900e+05,3.159800e+05,2.499900e+05
max,4.056016e+08,4.417477e+07,3.766584e+08,1.063836e+08,3.358785e+08


# Procesamiento para datasets geograficos

### Synthetizer functions

## Los niveles geograficos disponibles son:
 - Radios (RADIO_REF_ID)
 - Fracciones (IDFRAC, no es clave unica)
 - Dptos (DPTO)
 - Provs (PROV)
 - Aglos (AGLOMERADO)
 - (Region)

### Cargar geometrias

In [11]:
import geopandas as gpd

def compute_area_km2(geo_df):
    """Calcula el área en km^2 de un GeoDataFrame y lo añade como una nueva columna."""
    geo_df['area_km2'] = geo_df['geometry'].to_crs('epsg:3395').map(lambda p: p.area / 10**6)
    return geo_df

# Rutas de los archivos de polígonos
admin310_f = './../../geoespacial-censo-IGN/censos_shp_CONICET_dissolved/fracs_2010.shp'
admin210_f = './../../geoespacial-censo-IGN/censos_shp_CONICET_dissolved/dptos_2010.shp'
admin1_f = './../../geoespacial-censo-IGN/IGN_shp/ign_provincia'

# Cargar polígonos de provincias del IGN
admin1 = gpd.read_file(admin1_f)
admin1['PROV'] = admin1.IN1.astype(int)
admin1 = admin1[['PROV', 'geometry']]

# Cargar y procesar polígonos de fracciones de CONICET
admin310 = gpd.read_file(admin310_f)
admin310['IDFRAC'] = admin310.PROV_ + admin310.DEPTO_ + admin310.FRACC_ + '00'
admin310 = admin310[['IDFRAC', 'geometry']]

# Cargar y procesar polígonos de departamentos de CONICET
admin210 = gpd.read_file(admin210_f)
admin210['DPTO'] = (admin210['PROV_'] + admin210['DEPTO_']).astype(int)
admin210 = admin210[['DPTO', 'geometry']]

# Ajustar CRS para que todos los GeoDataFrames tengan el mismo CRS que admin1
admin210 = admin210.to_crs(admin1.crs)
admin310 = admin310.to_crs(admin1.crs)

# Calcular el área en km^2 para cada GeoDataFrame
admin1 = compute_area_km2(admin1)
admin210 = compute_area_km2(admin210)
admin310 = compute_area_km2(admin310)


/home/matias/anaconda3/envs/base2/lib/python3.11/site-packages/shapely/measurement.py:45: RuntimeWarning: invalid value encountered in area
  return lib.area(geometry, **kwargs)


### Funcion guardar geojson

In [12]:
import os
import geopandas as gpd

def save_geojson(gdf, filename='test.geojson'):
    if not os.path.exists('./../data/geojson/'):
        os.makedirs('./../data/geojson/')
    
    filepath = './../data/geojson/' + filename
    if os.path.exists(filepath):
        os.remove(filepath)  # Eliminar si el geojson existe, porque no se admite la sobrescritura
    
    gdf.to_file(filepath, driver="GeoJSON", encoding='utf-8')

def process_and_save(data, grouper, geo_df, filename_prefix, frac=0.05):
    # Sintetizar los datos
    df = sintetizar(data, [grouper], base=filename_prefix, frac=frac).drop('timestamp', axis=1)
    
    # Cambiar la forma del DataFrame
    df = df.set_index(list(df.drop('valor', axis = 1).columns)).unstack([0, 1])['valor']
    df.columns = ['_'.join(col) for col in df.columns.values]
    df = df.reset_index()
    
    # Fusionar con un GeoDataFrame
    gdf = gpd.GeoDataFrame(df.merge(geo_df), crs=geo_df.crs)
    
    # Guardar como GeoJSON
    save_geojson(gdf, filename=f'pobreza_{filename_prefix}_{grouper}.geojson')


In [13]:
# debug merge por provincia
# df[['NOMPROV', 'IDFRAC']].drop_duplicates().merge(admin310, on = ['IDFRAC'], how = 'outer', indicator = True).groupby('NOMPROV').agg({'_merge': 'value_counts'}).unstack()

In [21]:
# Define los GeoDataFrames para cada nivel geográfico
geo_dfs = {
    # 'PROV': admin1,
    # 'DPTO': admin210,
    'IDFRAC': admin310
}

# Define los subconjuntos de datos y sus prefijos correspondientes para los nombres de los archivos
data_subsets = {
    # 'P': cross_section_personas,
    # 'M24': cross_section_personas[cross_section_personas.P03 >= 24],
    'M14': cross_section_personas[cross_section_personas.P03 <= 14],
    # 'M6': cross_section_personas[cross_section_personas.P03 <= 6],
    # 'H': cross_section_hogares  # Asumiendo que data también incluye información de hogares
}

ow = False  # Sobrescribir archivos existentes
# Procesar y guardar los datos para cada subconjunto y nivel geográfico
for filename_prefix, subset in data_subsets.items():
    for grouper, geo_df in geo_dfs.items():
        print(f"Procesando subset '{filename_prefix}' a nivel geografico '{grouper}'...")
        
        if not ow and os.path.exists(f'pobreza_{filename_prefix}_{grouper}.geojson'): continue
        
        process_and_save(subset, grouper, geo_df, filename_prefix)
        print(f"Procesamiento de subset '{filename_prefix}' a nivel geografico '{grouper}' completado.")
    print(f"Procesamiento completo para subset '{filename_prefix}'.")
print("Todos los procesamientos han sido completados.")



Procesando subset 'M14' a nivel geografico 'IDFRAC'...
Procesamiento de subset 'M14' a nivel geografico 'IDFRAC' completado.
Procesamiento completo para subset 'M14'.
Todos los procesamientos han sido completados.


## Mapbox

In [15]:
import time


## Tilesets Upload

In [22]:
import os

# groups = ["H", "P", "M24", "M14", "M6"]
groups = ["M14"]

path = './../data'

# Setting the environment variable
os.environ["MAPBOX_ACCESS_TOKEN"] = "sk.eyJ1IjoibWF0dXRlaWdsZXNpYXMyIiwiYSI6ImNrb3lvMWZyajAxZncycG8ycnJkaTI1ZjYifQ.LXJGImmBgQtWWrNOC1wTcA"

year_month = str(pd.Timestamp.now().year) + str(pd.Timestamp.now().month).zfill(2)
# year_month = '202309c'

for group in groups:
    print(f"Processing group: {group}")
    
    # Delete sources and tileset
    os.system(f"tilesets delete-source -f matuteiglesias2 income_provs_{group}")
    os.system(f"tilesets delete-source -f matuteiglesias2 income_dptos_{group}")
    os.system(f"tilesets delete-source -f matuteiglesias2 income_fracs_{group}")
    time.sleep(3)
    
    # Upload sources
    os.system(f"tilesets upload-source matuteiglesias2 income_provs_{group} {path}/geojson/pobreza_{group}_PROV.geojson")
    os.system(f"tilesets upload-source matuteiglesias2 income_dptos_{group} {path}/geojson/pobreza_{group}_DPTO.geojson")
    os.system(f"tilesets upload-source matuteiglesias2 income_fracs_{group} {path}/geojson/pobreza_{group}_IDFRAC.geojson")
    time.sleep(3)
    
    # Create tilesets
    os.system(f"tilesets create matuteiglesias2.income_{group}{year_month} --recipe {path}/recipes/pobreza_{group}-recipe.json --name 'Ingresos y Pobreza - {group}'")
    time.sleep(3)
    os.system(f"tilesets publish matuteiglesias2.income_{group}{year_month}")


Processing group: M14
Source deleted.
upload progress
{"id": "mapbox://tileset-source/matuteiglesias2/income_fracs_M14", "files": 1, "source_size": 168181145, "file_size": 168181145}
{"message": "matuteiglesias2.income_M14202309 already exists"}
{"message": "Processing matuteiglesias2.income_M14202309", "jobId": "clmtq5vjs001p08lfdb8s1385"}



✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/matuteiglesias2.income_M14202309 or run tilesets job matuteiglesias2.income_M14202309 clmtq5vjs001p08lfdb8s1385 to view the status of your tileset.


## Mapbox Styles Update

- Fetch the existing style JSON file.
- Update the sources element in the style JSON.
- Upload the updated style JSON back to Mapbox.

### Plantilla base para Mapas de Pobreza

In [260]:
import json
import requests

# Step 1: Fetch the existing style JSON file
# Use the "Retrieve a style" endpoint to get the current style JSON 
# Replace {username} and {style_id} with your Mapbox account and style ID
username = 'matuteiglesias2'
style_id = 'clmtqn2h705bg01p7424h77y3'

access_token = 'sk.eyJ1IjoibWF0dXRlaWdsZXNpYXMyIiwiYSI6ImNrb3lvMWZyajAxZncycG8ycnJkaTI1ZjYifQ.LXJGImmBgQtWWrNOC1wTcA'
url = f"https://api.mapbox.com/styles/v1/{username}/{style_id}?access_token={access_token}"
response = requests.get(url)
template_json = response.json()
STYLE_TEMPLATE = './template01.json'
with open(STYLE_TEMPLATE, 'w') as f:
    json.dump(template_json, f)


In [178]:
template = json.load(open(STYLE_TEMPLATE))

In [179]:
template_json['sources']['composite']['url']

'mapbox://mapbox.mapbox-streets-v8,mapbox.mapbox-terrain-v2,matuteiglesias2.income_M24202309,matuteiglesias2.ejido'

### Wrappers para editar Plantilla

In [286]:
def set_name(style_json, new_name):
    modified_json = style_json.copy()
    modified_json['name'] = new_name
    return modified_json

def set_center_and_zoom(style_json, center, zoom):
    modified_json = style_json.copy()
    modified_json['center'] = center
    modified_json['zoom'] = zoom
    return modified_json

def update_source_url(style_json, new_part):
    modified_json = style_json.copy()
    url = modified_json['sources']['composite']['url']
    
    # Split the URL on commas
    parts = url.split(',')
    
    # Iterate over each part and look for the part that starts with "matuteiglesias2." and contains "income"
    for i, part in enumerate(parts):
        if part.startswith("matuteiglesias2.") and "income" in part:
            parts[i] = "matuteiglesias2." + new_part
    
    # Reconstruct the URL
    modified_json['sources']['composite']['url'] = ','.join(parts)
    print(modified_json['sources']['composite']['url'])
    return modified_json

## VARIABLE
def recursive_replace_variable(nested_list, var_changes):
    new_list = []
    for item in nested_list:
        if isinstance(item, list):
            # If the item is a list, recursively apply the replacement
            new_list.append(recursive_replace_variable(item, var_changes))
        elif item in var_changes:
            # If the item is a key in var_changes, replace it
            new_list.append(var_changes[item])
        else:
            # Otherwise, keep the original item
            new_list.append(item)
    return new_list


def wrapper_varchange(style_json, var_changes, layer_ids):
    modified_json = style_json.copy()

    for layer in modified_json['layers']:
        if layer['id'] in layer_ids:
            # Check and update 'fill-color' if present
            if 'fill-color' in layer.get('paint', {}):
                layer['paint']['fill-color'] = recursive_replace_variable(layer['paint']['fill-color'], var_changes)
            
            # Check and update 'text-field' if present
            if 'text-field' in layer.get('layout', {}):
                layer['layout']['text-field'] = recursive_replace_variable(layer['layout']['text-field'], var_changes)
    return modified_json


def replace_scale_colors_in_json(style_json, new_values, layer_ids):
    modified_json = style_json.copy()
    
    for layer in modified_json['layers']:
        if layer['id'] in layer_ids:
            for i, item in enumerate(layer['paint']['fill-color']):
                if isinstance(item, list) and item[0] == 'interpolate':
                    # Replace from the 3rd item onward, because the first three are ['interpolate', ['linear'], ['get', 'SOME_VARIABLE']]
                    layer['paint']['fill-color'][2][3:] = new_values
    return modified_json


import matplotlib.pyplot as plt
import matplotlib.colors

def get_colors_from_cmap(cmap_name, n_stops):
    """
    Get a list of colors in hex format from a given cmap.
    
    Parameters:
    - cmap_name (str): Name of the color map (e.g. "RdYlGn_r").
    - n_stops (int): Number of color stops you want.
    
    Returns:
    - List of colors in hex format.
    """
    cmap = plt.get_cmap(cmap_name)
    colors = [cmap(i / (n_stops-1)) for i in range(n_stops)]
    hex_colors = [matplotlib.colors.rgb2hex(color) for color in colors]
    return hex_colors


def apply_ARS_format_in_json(style_json, layer_ids, varname):
    modified_json = style_json.copy()

    for layer in modified_json['layers']:
        if layer['id'] in layer_ids:
            # Modify 'text-field' property
            layer['layout']['text-field'] = [
                "to-string",
                [
                    "*",
                    1000,
                    ["round", ["/", ["get", varname], 1000]]
                ]
            ]
            # # Modify 'text-size' property
            # layer['layout']['text-size'] = 12

            # # Modify 'text-opacity' and 'text-color' in 'paint' property
            # layer['paint']['text-opacity'] = [
            #     "interpolate",
            #     ["linear"],
            #     ["zoom"],
            #     9.9,
            #     0,
            #     10,
            #     0.8,
            #     21,
            #     0.8,
            #     22,
            #     0
            # ]
            # layer['paint']['text-color'] = "rgb(0, 0, 0)"

    return modified_json


In [252]:

year_month = str(pd.Timestamp.now().year) + str(pd.Timestamp.now().month).zfill(2)
# year_month = '202309c'

In [253]:
template_json['sources']['composite']['url']

'mapbox://mapbox.mapbox-streets-v8,mapbox.mapbox-terrain-v2,matuteiglesias2.income_H202309,matuteiglesias2.ejido'

In [287]:
import json
import requests
import pandas as pd

# Define necessary variables
username = 'matuteiglesias2'
access_token = 'sk.eyJ1IjoibWF0dXRlaWdsZXNpYXMyIiwiYSI6ImNrb3lvMWZyajAxZncycG8ycnJkaTI1ZjYifQ.LXJGImmBgQtWWrNOC1wTcA'
styleuploader ='sk.eyJ1IjoibWF0dXRlaWdsZXNpYXMyIiwiYSI6ImNsaHlnMjRrbjAwaHEzZ280Yzh3NTY1dW8ifQ.htipF1LHE9GXTUzXfhAwTw'

styles_info = pd.read_csv('./../data/info/mapbox_styles.csv') # Replace with the correct path to your CSV


# # Extract the prefix from the source name
# def get_prefix(source_name):
#     # Expected format: matuteiglesias2.income_{PREFIX}{YEAR}{MONTH}
#     # Extract the PREFIX part
#     prefix = source_name.split("income_")[1][:-6]  # Remove year and month
#     return prefix

new_style_ids_dict = {}
# Loop through the styles and update each one
for _, row in styles_info.iterrows():
    template_json = json.load(open(STYLE_TEMPLATE))
    print("\n\nProcessing style:", row['title'])
    # style_id = row['id']
    # url = f"https://api.mapbox.com/styles/v1/{username}/{style_id}?access_token={access_token}"
    # response = requests.get(url)

    style_json = template_json.copy()

    # remove to avoid conflict
    style_json.pop('created', None)
    style_json.pop('modified', None)

    # Use the wrapper functions
    style_json = set_name(style_json, row['title']);
    style_json = set_center_and_zoom(style_json, [-58.3872, -34.5852], 11.2)
    print(row['source'], row['variable'])
    style_json = update_source_url(style_json, row['source'])
    # print()    

    layer_ids = ['text fracs', 'text dptos', 'text provs', 'fill fracs', 'fill dptos', 'fill provs']
    style_json = wrapper_varchange(style_json, {"Indigencia_mean": row['variable']}, 
                                   layer_ids=layer_ids)

    ## COLORES y ESCALA
    fill_layer_ids = ['fill fracs', 'fill dptos', 'fill provs']
    n_stops = 10

    # Determine the new range based on the title
    if 'Pobreza' in row['title']:
        cmap = 'RdYlGn_r'
        linspace_values = np.linspace(0, 0.6, n_stops)
    elif 'Indigencia' in row['title']:
        cmap = 'RdYlGn_r'
        linspace_values = np.linspace(0, 0.25, n_stops)
    elif 'Canasta Alimentaria' in row['title']:
        cmap = 'Reds'
        linspace_values = np.linspace(0, 150000, n_stops)
    elif 'Canasta Total' in row['title']:
        cmap = 'Blues'
        linspace_values = np.linspace(0, 300000, n_stops)
    elif 'Ingreso mediano' in row['title']:
        cmap = 'Greens'
        linspace_values = np.linspace(0, 300000, n_stops)
    elif 'Ingreso Total Familiar' in row['title']:
        cmap = 'Greens'
        linspace_values = np.linspace(0, 600000, n_stops)

    # Round to .01 precision
    linspace_values = [round(value, 2) for value in linspace_values]

    if 'AR$' in row['title']:
        linspace_values = [round(value, -3) for value in linspace_values]

    color_changes = get_colors_from_cmap(cmap, n_stops)
    
    # Interleave scale values and colors
    new_values = [val for pair in zip(linspace_values, color_changes) for val in pair]

    # Update the style_json with the new scale and colors
    style_json = replace_scale_colors_in_json(style_json, new_values, fill_layer_ids)


    # AR$ format
    text_layer_ids = ['text provs', 'text dptos', 'text fracs']
    if "AR$" in row['title']:
        style_json = apply_ARS_format_in_json(style_json, text_layer_ids, row['variable'])


    ## Style Json Edit Completed. SHOW:
    for layer in style_json['layers']:
        if layer['id'] in ['fill fracs', 'fill dptos', 'fill provs']:
            print(layer['paint']['fill-color'])
        if layer['id'] in ['text fracs', 'text dptos', 'text provs']:
            print(layer['layout']['text-field'])

    # Upload the updated style JSON
        # Create new style
    response = requests.post(
        f"https://api.mapbox.com/styles/v1/{username}?access_token={styleuploader}",
        json=style_json
    )

    response_status_code = response.status_code

    if response_status_code == 201:
        style_id = response.json()['id']; new_style_ids_dict[row.name] = style_id; print(style_id)
        print(f"Style updated for {row['title']}.")
    else:
        print(f"Error updating style for {row['title']}. Status code: {response_status_code}")

    # Replace existing style (    if response_status_code == 200:)
    # response = requests.patch(
    #     f"https://api.mapbox.com/styles/v1/{username}/{style_id}?access_token={styleuploader}",
    #     json=style_data
    # )


    # Check if 'id' field is not empty
    if pd.notnull(row['id']):
        # Delete the old style
        delete_response = requests.delete(
            f"https://api.mapbox.com/styles/v1/{username}/{row['id']}?access_token={styleuploader}"
        )
        if delete_response.status_code == 204:
            print(f"Successfully deleted old style with ID {row['id']}.")
        else:
            print(f"Error deleting style with ID {row['id']}. Status code: {delete_response.status_code}")


    time.sleep(0.5)

# Add the new IDs to the dataframe
styles_info['id'] = styles_info.index.map(new_style_ids_dict)

# Save the updated dataframe to the CSV
styles_info.to_csv('./../data/info/mapbox_styles.csv', index=False)



Processing style: Pobreza (%). Menores de 6
income_M6202309 Pobreza_mean
mapbox://mapbox.mapbox-streets-v8,mapbox.mapbox-terrain-v2,matuteiglesias2.income_M6202309,matuteiglesias2.ejido
['case', ['>', ['get', 'Total_sum'], 500], ['interpolate', ['linear'], ['get', 'Pobreza_mean'], 0.0, '#006837', 0.07, '#219c52', 0.13, '#73c264', 0.2, '#b7e075', 0.27, '#e9f6a1', 0.33, '#feeda1', 0.4, '#fdbf6f', 0.47, '#f67a49', 0.53, '#da362a', 0.6, '#a50026'], '#bfbfbf']
['case', ['>', ['get', 'Total_sum'], 2000], ['interpolate', ['linear'], ['get', 'Pobreza_mean'], 0.0, '#006837', 0.07, '#219c52', 0.13, '#73c264', 0.2, '#b7e075', 0.27, '#e9f6a1', 0.33, '#feeda1', 0.4, '#fdbf6f', 0.47, '#f67a49', 0.53, '#da362a', 0.6, '#a50026'], 'hsla(0, 0%, 31%, 0.3)']
['case', ['>', ['get', 'Total_sum'], 2000], ['interpolate', ['linear'], ['get', 'Pobreza_mean'], 0.0, '#006837', 0.07, '#219c52', 0.13, '#73c264', 0.2, '#b7e075', 0.27, '#e9f6a1', 0.33, '#feeda1', 0.4, '#fdbf6f', 0.47, '#f67a49', 0.53, '#da362a', 0.

### Actualiza estilos en la pagina web

In [288]:
import pandas as pd
import re

def update_style_id_in_file(filename, new_style_id):
    with open(filename, 'r') as file:
        file_contents = file.read()
        
    # Use a regular expression to replace style ID in the file content
    pattern = r"style: 'mapbox://styles/matuteiglesias2/[^']*'"
    replacement = f"style: 'mapbox://styles/matuteiglesias2/{new_style_id}'"
    new_content = re.sub(pattern, replacement, file_contents)
    
    with open(filename, 'w') as file:
        file.write(new_content)

# 1. Read the data file
styles_info = pd.read_csv('./../data/info/mapbox_styles.csv')

# 2. Iterate over each map HTML file you want to update
web_path = './../../../Documents/link/Pobreza/Mapas/maps/mbox/'
for _, row in styles_info.iterrows():
    map_html_file = web_path + row['map_html']
    new_style_id = row['id']
    
    # 3 & 4. Update the style ID in the file
    update_style_id_in_file(map_html_file, new_style_id)


In [289]:
pd.read_csv('./../data/info/mapbox_styles.csv')

,title,id,source,variable,page_html,map_html
0,Pobreza (%). Menores de 6,clmv3jfs905os01qrh4225j3j,income_M6202309,Pobreza_mean,men6porc.html,test_m6_pobreza.html
1,Pobreza (%). Menores de 14,clmv3jhe205jx01ma0sy0dt6i,income_M14202309,Pobreza_mean,men14porc.html,test_m14_pobreza.html
2,Pobreza (%). Mayores de 24,clmv3jj5c05ot01qr9xai56zr,income_M24202309,Pobreza_mean,may24porc.html,test_m24_pobreza.html
3,Pobreza (%). Personas,clmv3jmpm05l101qidm3sg6rk,income_P202309,Pobreza_mean,personasporc.html,test_pers_pobreza.html
4,Pobreza (%). Hogares,clmv3jort05nc01ns5s48ebp0,income_H202309,Pobreza_mean,hogaresporc.html,test_hogar_pobreza.html
5,Indigencia (%). Menores de 6,clmv3jqha05ny01p9fiulg7yv,income_M6202309,Indigencia_mean,men6indporc.html,test_m6_indigencia.html
6,Indigencia (%). Personas,clmv3jsc205ou01qrhdoi373l,income_P202309,Indigencia_mean,personasindporc.html,test_personas_indigencia.html
7,Indigencia (%). Mayores de 24,clmv3jv2m05nz01p95sff7rqi,income_M24202309,Indigencia_mean,may24indporc.html,test_m24_indigencia.html
8,Indigencia (%). Hogares,clmv3jxjv05jy01ma3fu27o76,income_H202309,Indigencia_mean,hogaresindporc.html,test_hogares_indigencia.html
9,Indigencia (%). Menores de 14,clmv3jzbh05nd01nsf7mr1gbf,income_M14202309,Indigencia_mean,men14indporc.html,test_m14_indigencia.html
